In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 1600, done.
remote: Counting objects: 100% (449/449), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 1600 (delta 353), reused 364 (delta 284), pack-reused 1151
Receiving objects: 100% (1600/1600), 110.20 MiB | 27.50 MiB/s, done.
Resolving deltas: 100% (1183/1183), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 25.7 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv


In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_SVDpp_Cython, MatrixFactorization_AsySVD_Cython, MatrixFactorization_WARP_Cython
from Recommenders.MatrixFactorization.PyTorch.MF_MSE_PyTorch import MF_MSE_PyTorch

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_mf_funk.db',
    'copy_prev_best_params': False,
    'save_github': True
}

In [10]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_mf_funk.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

In [11]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [12]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [13]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

In [14]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

def objective_function_bpr_mf(trial):
    early_stopping_params = {
        "validation_every_n": 5,
        "stop_on_validation": True,
        "evaluator_object": evaluator,
        "lower_validations_allowed": 10,
        "validation_metric": "MAP",
        "epochs": 1000
    }
                          
    recommender_instance = MatrixFactorization_SVDpp_Cython(URM_train)
    recommender_instance.fit(
        num_factors = trial.suggest_int("num_factors", 1, 500, log=True),
        sgd_mode = trial.suggest_categorical("sgd_mode", ["sgd", "adagrad", "adam", "rmsprop"]),
        batch_size = trial.suggest_categorical("batch_size", [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]),
        user_reg =  trial.suggest_float("user_reg", 1e-6, 1, log=True),
        item_reg = trial.suggest_float("item_reg", 1e-6, 1, log=True),
        learning_rate = trial.suggest_float("learning_rate", 1e-9, 1e-1, log=True),
        dropout_quota = trial.suggest_float("dropout_quota", 1e-3, 0.7, log=True),
        init_std_dev = trial.suggest_float("init_std_dev", 1e-2, 10, log=True),
        use_bias=False,
        **early_stopping_params
    )
    
    epochs = recommender_instance.get_early_stopping_final_epochs_dict()["epochs"]
    trial.set_user_attr("epochs", epochs) 
        
    result_df, _ = evaluator.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10, "MAP"]

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions


In [15]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_funk_svd',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective_function_bpr_mf, n_trials=15)

[I 2023-12-02 23:59:17,782] Using an existing study with name 'hyperparameters_tuning_funk_svd' instead of creating a new one.


MatrixFactorization_SVDpp_Cython_Recommender: URM Detected 627 ( 4.8%) users with no interactions.
MatrixFactorization_SVDpp_Cython_Recommender: URM Detected 247 ( 1.1%) items with no interactions.
SVD++: Processed 382992 (100.0%) in 0.13 sec. MSE loss 9.36E-01. Sample per second: 2851821
SVD++: Epoch 1 of 1000. Elapsed time 0.13 sec
SVD++: Processed 382992 (100.0%) in 0.25 sec. MSE loss 7.74E-01. Sample per second: 1548677
SVD++: Epoch 2 of 1000. Elapsed time 0.24 sec
SVD++: Processed 382992 (100.0%) in 0.37 sec. MSE loss 6.43E-01. Sample per second: 1048176
SVD++: Epoch 3 of 1000. Elapsed time 0.36 sec
SVD++: Processed 382992 (100.0%) in 0.48 sec. MSE loss 5.43E-01. Sample per second: 800790
SVD++: Epoch 4 of 1000. Elapsed time 0.47 sec
SVD++: Processed 382992 (100.0%) in 0.59 sec. MSE loss 4.66E-01. Sample per second: 648982
SVD++: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 8.87 sec. Users per second: 1179
SVD++: CUTOFF: 10 - PRECISION: 0.0191469, PRECISION_R

[I 2023-12-03 00:01:10,649] Trial 15 finished with value: 0.005968347330005685 and parameters: {'num_factors': 1, 'sgd_mode': 'rmsprop', 'batch_size': 8, 'user_reg': 0.0001856900773944396, 'item_reg': 0.00013210330191934392, 'learning_rate': 0.002691914665583863, 'dropout_quota': 0.003030850403152191, 'init_std_dev': 0.010207513538780985}. Best is trial 2 with value: 0.011808883939956966.


MatrixFactorization_SVDpp_Cython_Recommender: URM Detected 627 ( 4.8%) users with no interactions.
MatrixFactorization_SVDpp_Cython_Recommender: URM Detected 247 ( 1.1%) items with no interactions.
SVD++: Processed 382986 (100.0%) in 1.42 sec. MSE loss 1.00E+00. Sample per second: 268715
SVD++: Epoch 1 of 1000. Elapsed time 0.57 sec
SVD++: Processed 382986 (100.0%) in 1.00 sec. MSE loss 1.00E+00. Sample per second: 383792
SVD++: Epoch 2 of 1000. Elapsed time 1.15 sec
SVD++: Processed 382986 (100.0%) in 0.57 sec. MSE loss 1.00E+00. Sample per second: 675870
SVD++: Epoch 3 of 1000. Elapsed time 1.72 sec
SVD++: Processed 382986 (100.0%) in 1.14 sec. MSE loss 1.00E+00. Sample per second: 335934
SVD++: Epoch 4 of 1000. Elapsed time 2.29 sec
SVD++: Processed 382986 (100.0%) in 0.71 sec. MSE loss 1.00E+00. Sample per second: 537037
SVD++: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 9.23 sec. Users per second: 1132
SVD++: CUTOFF: 10 - PRECISION: 0.0005451, PRECISION_RECA

[I 2023-12-03 00:03:33,080] Trial 16 finished with value: 0.00015856696542427222 and parameters: {'num_factors': 12, 'sgd_mode': 'sgd', 'batch_size': 2, 'user_reg': 9.83033672192201e-06, 'item_reg': 0.004180261448385354, 'learning_rate': 1.2862724587763887e-05, 'dropout_quota': 0.00836951511222294, 'init_std_dev': 0.06293168562046117}. Best is trial 2 with value: 0.011808883939956966.


...

[I 2023-12-03 02:08:35,900] Trial 27 finished with value: 0.011314890212652213 and parameters: {'num_factors': 102, 'sgd_mode': 'sgd', 'batch_size': 4, 'user_reg': 1.850867056021116e-05, 'item_reg': 0.000397632113364876, 'learning_rate': 0.008197886512044696, 'dropout_quota': 0.0010277821260400888, 'init_std_dev': 0.029333026901048283}. Best is trial 2 with value: 0.011808883939956966.


MatrixFactorization_SVDpp_Cython_Recommender: URM Detected 627 ( 4.8%) users with no interactions.
MatrixFactorization_SVDpp_Cython_Recommender: URM Detected 247 ( 1.1%) items with no interactions.
SVD++: Processed 382988 (100.0%) in 4.16 sec. MSE loss 9.94E-01. Sample per second: 91992
SVD++: Epoch 1 of 1000. Elapsed time 3.93 sec
SVD++: Processed 382988 (100.0%) in 4.09 sec. MSE loss 9.81E-01. Sample per second: 93698
SVD++: Epoch 2 of 1000. Elapsed time 7.86 sec
SVD++: Processed 382988 (100.0%) in 4.08 sec. MSE loss 9.66E-01. Sample per second: 93881
SVD++: Epoch 3 of 1000. Elapsed time 11.85 sec
SVD++: Processed 382988 (100.0%) in 4.01 sec. MSE loss 9.46E-01. Sample per second: 95573
SVD++: Epoch 4 of 1000. Elapsed time 15.78 sec
SVD++: Processed 382988 (100.0%) in 3.89 sec. MSE loss 9.22E-01. Sample per second: 98408
SVD++: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 9.38 sec. Users per second: 1115
SVD++: CUTOFF: 10 - PRECISION: 0.0119740, PRECISION_RECALL_

[I 2023-12-03 02:15:07,213] Trial 28 finished with value: 0.010396753288155336 and parameters: {'num_factors': 102, 'sgd_mode': 'sgd', 'batch_size': 4, 'user_reg': 1.914719245929249e-05, 'item_reg': 0.00042625661140988606, 'learning_rate': 0.016451945326606498, 'dropout_quota': 0.0018505635267696535, 'init_std_dev': 0.035797555229398356}. Best is trial 2 with value: 0.011808883939956966.


MatrixFactorization_SVDpp_Cython_Recommender: URM Detected 627 ( 4.8%) users with no interactions.
MatrixFactorization_SVDpp_Cython_Recommender: URM Detected 247 ( 1.1%) items with no interactions.
SVD++: Processed 382988 (100.0%) in 1.52 sec. MSE loss 9.96E-01. Sample per second: 252045
SVD++: Epoch 1 of 1000. Elapsed time 1.08 sec
SVD++: Processed 382988 (100.0%) in 1.58 sec. MSE loss 9.84E-01. Sample per second: 242741
SVD++: Epoch 2 of 1000. Elapsed time 2.14 sec
SVD++: Processed 382988 (100.0%) in 1.64 sec. MSE loss 9.72E-01. Sample per second: 233622
SVD++: Epoch 3 of 1000. Elapsed time 3.20 sec
SVD++: Processed 382988 (100.0%) in 1.70 sec. MSE loss 9.59E-01. Sample per second: 224951
SVD++: Epoch 4 of 1000. Elapsed time 4.26 sec
SVD++: Processed 382988 (100.0%) in 1.77 sec. MSE loss 9.45E-01. Sample per second: 216734
SVD++: Validation begins...
EvaluatorHoldout: Processed 10456 (100.0%) in 9.15 sec. Users per second: 1143
SVD++: CUTOFF: 10 - PRECISION: 0.0032708, PRECISION_RECA

[I 2023-12-03 02:18:47,742] Trial 29 finished with value: 0.007222749754071469 and parameters: {'num_factors': 24, 'sgd_mode': 'sgd', 'batch_size': 4, 'user_reg': 0.00010526432406818803, 'item_reg': 0.0015993386383908906, 'learning_rate': 0.009756776652236353, 'dropout_quota': 0.00480007070287898, 'init_std_dev': 0.09542347414353596}. Best is trial 2 with value: 0.011808883939956966.


In [16]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [17]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [18]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [19]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_funk_svd.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_funk_svd.json', 
            'tuning_results/best_params_funk_svd.json',
            'Funk SVD tuning results (from kaggle notebook)'
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_funk_svd.json', 
        '/kaggle/working/best_params_funk_svd.json'
    )

In [20]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        'tuning_results/tuning_mf_funk.db',
        'Tuning db updated results (from kaggle notebook)'
    )